# Liquid Alpha (Consensus based weights)

This notebook accompanies the release of the consensus-based weight feature. The weight we are referring to here is the alpha term in the moving average portion of the bond calculation. See the medium post for a detailed discussion of the reasoning behind this update.

Here, we propose another approach in addition to the commitment scheme to amplify the advantage of validators that take actions earlier. In turn, this amplifies the disadvantage of validators who copy or act reactively to other validators' actions. Once this disadvantage reaches a certain threshold, it becomes preferable for TAO owners to either perform miner-evaluation work as intended or delegate their stakes to other validators who perform such work.

The rest of this notebook shows a method subnet owners can use to determine their preferred value of `alpha_low, alpha_high, commit_reveal_interval`

Please refer to our paper or [blog post](https://blog.bittensor.com/consensus-based-weights-1c5bbb4e029b) for the description of the algorithm.

## Imports & setup

In [1]:
import os 
import pickle

import torch 
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import bittensor as bt

from experiment_setup import ExperimentSetup

setup = ExperimentSetup(
    processes = 1, # processes to run with 
    liquid_alpha = True, 
)

## Download metagraphs

In [ ]:
from download_metagraphs import DownloadMetagraph
DownloadMetagraph(setup = setup).run()

## Run simulation

In [ ]:
from weight_copy_simulation import WeightCopySimulation
WeightCopySimulation(setup = setup).run_simulation()

## Analysis
(1) Calculate relative dividend rate for weight copier

(2) Choose the optimal setting that gives weight copier the most discound in dividend 

(3) Check how would this setting affect the dividend of regular honest vlaidators 
 

### (1) Calculate relative dividend rate for weight copier under different setting

With $D$ as dividend; $S$ as stake; $\mathcal Z$ as the set of validators.
We use the relative dividend rate of the copier $j$,

$$G^j = \frac{D^j/S^j}{\underset{i \in \mathcal Z \setminus \{j\}}{\mathrm{median}} \{D^i/S^i\}}$$

- The lower the dividend rate (G), the more discount in dividend we are giving to the weight copier

- 360 * conceal_period = commit_reveal_weight_interval

- commit_reveal_weight_interval, alpha_low and alpha_high are the parameters made availble for the SN owner to set to the chain

In [2]:
def get_relative_dividend_rate(setup):
    div_rates = []

    for netuid in setup.netuids:
        for conceal_period in setup.conceal_periods:
            for alpha_low in setup.alpha_lows:
                for alpha_high in setup.alpha_highs:
                    if alpha_low > alpha_high:
                        continue
                    
                    try: 
                        with open(f"{setup.result_path}/yuma_result_netuid{netuid}_conceal{conceal_period}_al{alpha_low:.1f}_ah{alpha_high:.1f}.pkl", 'rb') as handle:
                            _yuma_results = pickle.load(handle)

                        dividend = [
                            (s["validator_reward_normalized"] / s["stake"]).tolist()
                            for idx, s in _yuma_results.items()
                        ]

                        dividend_df = pd.DataFrame(
                            dividend,
                            columns=[f"v{i}" for i in range(len(dividend[0]) - 1 )] + ["v_bad"],
                        )
                        
                        div_last = dividend_df.iloc[-1]
                        if (div_last.isna()).any():
                            div_rate = None
                        else:
                            div_rate = div_last[-1] / div_last[:-1].median() 

                        div_rates.append([netuid, conceal_period, alpha_low, alpha_high, div_rate, div_last])
                    
                    except:
                        div_rates.append([netuid, conceal_period, alpha_low, alpha_high, None, None])


    div_rates = pd.DataFrame(div_rates, dtype='float64', columns = ['netuid', 'conceal_period', 'alpha_low', 'alpha_high', 'G', 'dividend'])
    div_rates.index = div_rates.index.map(lambda x : x)

    return div_rates

In [3]:
div_rates = get_relative_dividend_rate(setup)
div_rates.sort_values('G')

/tmp/ipykernel_1267731/2707014565.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  div_rates = get_relative_dividend_rate(setup)


,netuid,conceal_period,alpha_low,alpha_high,G,dividend
1470,11.0,15.0,0.1,0.1,0.853703,v0 1.136394 v1 1.136305 v2 0...
1475,11.0,15.0,0.3,0.3,0.860689,v0 1.131615 v1 1.129273 v2 0...
1471,11.0,15.0,0.1,0.3,0.861587,v0 1.131760 v1 1.129525 v2 0...
1395,11.0,5.0,0.1,0.1,0.867577,v0 1.126479 v1 1.126087 v2 0...
1396,11.0,5.0,0.1,0.3,0.874954,v0 1.114025 v1 1.105638 v2 0...
...,...,...,...,...,...,...
4990,37.0,15.0,0.5,0.7,NaN,None
4991,37.0,15.0,0.5,0.9,NaN,None
4992,37.0,15.0,0.7,0.7,NaN,None
4993,37.0,15.0,0.7,0.9,NaN,None


### (2) Choose the optimal setting that gives weight copier the most discound in dividend

- The lower the relative dividend rate (G), the more discount in dividend we are giving to the weight copier


- commit_reveal_weight_interval = 360 * conceal_period


- For the conceal period to be effective, you should set a conceal period large enough to produce enough lost in dividend for the weight copier.

| Dividend gain (G) | Effect                                                                                                     |
|------------------|------------------------------------------------------------------------------------------------------------|
| < 1               | Norminator lost the incentive to delegate to weight copier, weight copier earn less validator take.  |
| < 0.82          | Weigh copier lost the incentive to copy weight.                                                      |

- If given a conceal period long enough (>15 hours) and the SN still fail to produce enough lost in dividend, it means that there is not enough churn and weight movement in the SN, so the existing weight copiy fix may not work for your SN. Depending on the situation, you may choose to increase competitiveness/ churn in your SN or just leave the weight copier as is. Cause when there is no churn in the SN, there would be no movement in consensus as well, so the weight copier would not be as beneficial. 

- Note that when the conceal period was set too long, it would slow down the discovery of new miners, putting them at risk for deregistration. Further more, it would means that any change in the network would only be observable after 360 * conceal_period blocks. 

In [4]:
div_rates['best_param'] = False

for netuid in setup.netuids:
    df = div_rates[div_rates.netuid == netuid]
    df = df[df.G == df.G.min()] # that gives the lowest G 
    df = df[df.conceal_period == df.conceal_period.min()] # that minimize conceal period
    df = df[df.alpha_high == df.alpha_high.max()] # that maximize alpha_high
    df = df[df.alpha_low == df.alpha_low.max()] # that maximize alpha_low
    div_rates.loc[df.index, 'best_param'] = True

best_params = div_rates[div_rates.best_param == True]
best_params

,netuid,conceal_period,alpha_low,alpha_high,G,dividend,best_param
14,1.0,0.0,0.9,0.9,1.017643,v0 9.768890e-01 v1 9.965321e-01 v2...,True
269,2.0,15.0,0.9,0.9,0.978237,v0 0.999433 v1 1.010608 v2 0...,True
390,3.0,15.0,0.1,0.1,1.050416,v0 0.759324 v1 1.214824 v2 0...,True
529,4.0,15.0,0.1,0.9,1.273482,v0 0.930005 v1 1.087235 v2 1...,True
615,5.0,9.0,0.1,0.1,0.997380,v0 0.993586 v1 0.999255 v2 0...,True
936,7.0,15.0,0.3,0.5,0.977437,v0 1.003460 v1 1.005339 v2 1...,True
1079,8.0,15.0,0.9,0.9,1.009575,v0 1.109204 v1 1.082475 v2 1...,True
1336,10.0,15.0,0.1,0.3,0.967946,v0 1.007580 v1 0.996352 v2 0...,True
1470,11.0,15.0,0.1,0.1,0.853703,v0 1.136394 v1 1.136305 v2 0...,True
1619,12.0,15.0,0.9,0.9,0.963875,v0 0.994135 v1 0.997764 v2 0...,True


### (3) Check ther performance with the selected parameter and how would the setting affects the dividend of honest vlaidators  

#### Where would weight copier be positioned compared to honest validator in terms of dividend
- The lower the quentile the better

In [5]:
for idx, row in best_params.iterrows(): 
    div = torch.tensor(list(row.dividend.values))
    quantile = (div < div[-1]).sum()/len(div)
    best_params.loc[idx, 'quantile'] = quantile.item()

best_params

/usr/lib/python3/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,netuid,conceal_period,alpha_low,alpha_high,G,dividend,best_param,quantile
14,1.0,0.0,0.9,0.9,1.017643,v0 9.768890e-01 v1 9.965321e-01 v2...,True,0.608696
269,2.0,15.0,0.9,0.9,0.978237,v0 0.999433 v1 1.010608 v2 0...,True,0.181818
390,3.0,15.0,0.1,0.1,1.050416,v0 0.759324 v1 1.214824 v2 0...,True,0.640000
529,4.0,15.0,0.1,0.9,1.273482,v0 0.930005 v1 1.087235 v2 1...,True,0.750000
615,5.0,9.0,0.1,0.1,0.997380,v0 0.993586 v1 0.999255 v2 0...,True,0.478261
936,7.0,15.0,0.3,0.5,0.977437,v0 1.003460 v1 1.005339 v2 1...,True,0.045455
1079,8.0,15.0,0.9,0.9,1.009575,v0 1.109204 v1 1.082475 v2 1...,True,0.555556
1336,10.0,15.0,0.1,0.3,0.967946,v0 1.007580 v1 0.996352 v2 0...,True,0.000000
1470,11.0,15.0,0.1,0.1,0.853703,v0 1.136394 v1 1.136305 v2 0...,True,0.238095
1619,12.0,15.0,0.9,0.9,0.963875,v0 0.994135 v1 0.997764 v2 0...,True,0.090909


#### How would the dividend of honest validator change with or without liquid alpha

- The goal here is to make sure any parameter we are choosing here would not decrease the dividend that honest peers are receiving. 

- Note that when alpha_low = alpha_high = 0.9, it is equivalent to when liquid alpha is disabled

- We can consider a success when the MSE is low

In [7]:
import torch.nn as nn
loss = nn.MSELoss()

best_params['MSE'] = None
for idx, row in best_params.iterrows(): 
    original_div = div_rates[(div_rates.netuid == row.netuid) & (div_rates.conceal_period == row.conceal_period) & (div_rates.alpha_high == 0.9) & (div_rates.alpha_low == 0.9)]
    original_div = torch.tensor(list(original_div.dividend.values))[0]
    la_div = torch.tensor(list(row.dividend.values))
    best_params.loc[idx, 'MSE'] = loss(original_div, la_div).item()

best_params

/tmp/ipykernel_1267731/3916975874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_params['MSE'] = None
/usr/lib/python3/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,netuid,conceal_period,alpha_low,alpha_high,G,dividend,best_param,quantile,MSE
14,1.0,0.0,0.9,0.9,1.017643,v0 9.768890e-01 v1 9.965321e-01 v2...,True,0.608696,0.0
269,2.0,15.0,0.9,0.9,0.978237,v0 0.999433 v1 1.010608 v2 0...,True,0.181818,0.0
390,3.0,15.0,0.1,0.1,1.050416,v0 0.759324 v1 1.214824 v2 0...,True,0.640000,0.007146
529,4.0,15.0,0.1,0.9,1.273482,v0 0.930005 v1 1.087235 v2 1...,True,0.750000,0.002072
615,5.0,9.0,0.1,0.1,0.997380,v0 0.993586 v1 0.999255 v2 0...,True,0.478261,0.000142
936,7.0,15.0,0.3,0.5,0.977437,v0 1.003460 v1 1.005339 v2 1...,True,0.045455,0.000006
1079,8.0,15.0,0.9,0.9,1.009575,v0 1.109204 v1 1.082475 v2 1...,True,0.555556,0.0
1336,10.0,15.0,0.1,0.3,0.967946,v0 1.007580 v1 0.996352 v2 0...,True,0.000000,0.000007
1470,11.0,15.0,0.1,0.1,0.853703,v0 1.136394 v1 1.136305 v2 0...,True,0.238095,0.006796
1619,12.0,15.0,0.9,0.9,0.963875,v0 0.994135 v1 0.997764 v2 0...,True,0.090909,0.0


In [11]:
df = best_params[best_params.alpha_low != 0.9]
df = df[df.MSE < 0.001]

df

,netuid,conceal_period,alpha_low,alpha_high,G,dividend,best_param,quantile,MSE
615,5.0,9.0,0.1,0.1,0.997380,v0 0.993586 v1 0.999255 v2 0...,True,0.478261,0.000142
936,7.0,15.0,0.3,0.5,0.977437,v0 1.003460 v1 1.005339 v2 1...,True,0.045455,0.000006
1336,10.0,15.0,0.1,0.3,0.967946,v0 1.007580 v1 0.996352 v2 0...,True,0.000000,0.000007
2284,17.0,15.0,0.1,0.9,0.946778,v0 1.015642 v1 1.026432 v2 0...,True,0.217391,0.000059
2712,21.0,0.0,0.7,0.7,0.999999,v0 0.999791 v1 0.999967 v2 0...,True,0.333333,0.000003
3199,24.0,11.0,0.1,0.9,0.997289,v0 1.002180 v1 0.984031 v2 1...,True,0.428571,0.00002
3630,27.0,15.0,0.1,0.1,0.990384,v0 0.852913 v1 1.003531 v2 0...,True,0.160000,0.000029
4170,31.0,15.0,0.1,0.1,0.934582,v0 1.024073 v1 1.024509 v2 1...,True,0.181818,0.000067
4414,33.0,11.0,0.1,0.9,0.988717,v0 0.818151 v1 1.000240 v2 1...,True,0.052632,0.000124
